In [4]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

import time

from projectutils import read_data 
import sys


In [5]:
import os
import pandas as pd
import pyarrow.parquet as pq

def generate_parquet_summary(directory):
    # Lista todos os arquivos no diretório
    files = [f for f in os.listdir(directory) if f.endswith('.parquet')]
    
    # Cria listas para armazenar os nomes dos arquivos e a contagem de linhas
    file_names = []
    line_counts = []
    
    # Itera sobre cada arquivo, conta as linhas e armazena as informações
    for file in files:
        file_path = os.path.join(directory, file)
        parquet_file = pq.ParquetFile(file_path)
        file_names.append(file)
        line_counts.append(parquet_file.metadata.num_rows)
    
    # Cria o DataFrame
    summary_df = pd.DataFrame({
        'file_name': file_names,
        'line_count': line_counts
    })
    
    return summary_df



def calculate_group_ids(column):
    # Inicializa listas para soma cumulativa e IDs
    stopped_cumsum = []
    group_ids = []
    
    # Variáveis auxiliares
    cumsum = 0
    group_id = 1
    
    # Itera pelos valores da coluna
    for value in column:
        if cumsum + value > 1000000:
            group_id += 1
            cumsum = 0
        
        cumsum += value
        stopped_cumsum.append(cumsum)
        group_ids.append(group_id)
        
        if cumsum == 5:
            cumsum = 0
            group_id += 1
    
    return pd.Series(group_ids, index=column.index)


def create_agg_nomes_agrupados():
    # Exemplo de uso
    directory = 'data/nomes_agrupados'
    df = generate_parquet_summary(directory)
    df = df.sort_values('line_count')
    df['id'] = calculate_group_ids(df['line_count'])
    df['cod_muni'] = df['file_name'].apply(lambda filename: filename.split('_')[1].split('.')[0])

    import json
    dict_files = df.groupby('id')['cod_muni'].apply(lambda x: list(x)).to_dict()
    return dict_files

In [8]:
df = read_data('main_data_2024-03-31')

MemoryError: Unable to allocate 3.25 GiB for an array with shape (9, 48498930) and data type object

In [9]:
from pyarrow.parquet import ParquetFile
import pyarrow as pa 

pf = ParquetFile('data/') 
first_ten_rows = next(pf.iter_batches(batch_size = 10)) 
df = pa.Table.from_batches([first_ten_rows]).to_pandas()

In [10]:
df

,reference_month,subs_asterisk,nome_master,nome_muni,uf,cpf,cpf_brasil,cnpj,numero_inicio,cod_muni,merchant_tax_id,mmhid_merge,merchant_market_hierarchy_id
0,2024-04-30,SumUp,ANDRELUISMO,São João da Boa Vista,SP,None,None,31632735000161,None,3549102,None,<NA>,<NA>
1,2024-05-31,SumUp,GABRIELFERREI,Patos de Minas,MG,None,None,None,None,3148004,None,<NA>,<NA>
2,2024-05-31,SumUp,SMARTSHOP,Belo Jardim,PE,None,None,None,None,2601706,None,<NA>,<NA>
3,2024-03-31,SumUp,SIONLINE,São Paulo,SP,None,None,None,None,3550308,None,<NA>,<NA>
4,2024-03-31,SumUp,FARMACIASANT,Igreja Nova,AL,None,None,None,None,2703205,None,<NA>,<NA>
5,2024-05-31,SumUp,FEIRATEMDE,Santa Cruz,PB,None,None,None,None,2513208,None,<NA>,<NA>
6,2024-03-31,SumUp,PIVASGAMES,São Paulo,SP,None,None,None,None,3550308,None,<NA>,<NA>
7,2024-03-31,SumUp,CARMONAHAIR,São Paulo,SP,None,None,None,None,3550308,None,<NA>,<NA>
8,2024-03-31,SumUp,TRFERREIRA,São Paulo,SP,None,None,None,None,3550308,None,<NA>,<NA>
9,2024-05-31,SumUp,TIAOFOTOGRAF,São Paulo,SP,None,None,None,None,3550308,None,<NA>,<NA>


In [11]:
df.columns

Index(['reference_month', 'subs_asterisk', 'nome_master', 'nome_muni', 'uf',
       'cpf', 'cpf_brasil', 'cnpj', 'numero_inicio', 'cod_muni',
       'merchant_tax_id', 'mmhid_merge', 'merchant_market_hierarchy_id'],
      dtype='object')

In [6]:
df = pd.DataFrame(create_agg_nomes_agrupados(), columns = ['ixd', 'cod_muni'])

In [8]:
df = create_agg_nomes_agrupados()

In [10]:
[k for k, v in df.items() if '3550308' in v]

[18]

In [48]:
df = pd.read_parquet('data/intermediary/part_18.parquet')

In [49]:
df

,subs_asterisk,nome_master,nome_muni,uf,cpf,cpf_brasil,qtd_cpfs,qtd_cnpjs,cnpj,numero_inicio,cod_muni,merchant_tax_id,mmhid_merge,merchant_market_hierarchy_id,id_ton,nome_ton,inicio,resultado_names,agrupamento_nome_1,_merge
0,SumUp,REYTECH,São Paulo,SP,None,None,<NA>,1,55288394000117,None,3550308,None,<NA>,<NA>,NaN,None,REYTEC,[REYTECH],REYTECH3550308,both
1,SumUp,APPSERVIDORE,São Paulo,SP,None,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,NaN,None,APPSER,[APPSERVIDORE],APPSERVIDORE3550308,both
2,SumUp,JOSEWILSONP,São Paulo,SP,17782026830,None,26,10,20701303000149,None,3550308,None,<NA>,<NA>,NaN,None,JOSEWI,[JOSEWILSONP],"JOSEWILSONP, JOSEWILSONPEGADO3550308",both
3,SumUp,MARIAARTS,São Paulo,SP,None,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,NaN,None,MARIAA,[MARIAARTS],MARIAARTS3550308,both
4,SumUp,CANILBUTDOGG,São Paulo,SP,None,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,NaN,None,CANILB,[CANILBUTDOGG],CANILBUTDOGG3550308,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334900,Ton,CLINICAODONTOLOGIADENISEFERES,São Paulo,SP,None,None,<NA>,<NA>,03670613000124,None,3550308,None,<NA>,<NA>,1818529.0,CLINICA ODONTOLOGIA DENISE FERES,CLINIC,[CLINICAODONTOLOGIADENISEFERES],"CLINICAODONTOLOGIA, CLINICAODONTOLOGIADENISEFE...",both
1334901,Ton,POINTDOCOYOTE,São Paulo,SP,16154481870,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,1818530.0,POINT DO COYOTE,POINTD,[POINTDOCOYOTE],POINTDOCOYOTE3550308,both
1334902,Ton,ANGELICADAMOTALIMA,São Paulo,SP,44972567821,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,1818554.0,ANGELICA DA MOTA LIMA,ANGELI,[ANGELICADAMOTALIMA],ANGELICADAMOTALIMA3550308,both
1334903,Ton,BRUNACECCHETTIESTETICA,São Paulo,SP,39824084827,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,1818555.0,BRUNA CECCHETTI ESTETICA,BRUNAC,[BRUNACECCHETTIESTETICA],BRUNACECCHETTIESTETICA3550308,both


In [50]:
from group_tam_id import (
    assign_group_ids,
    open_ton, load_to_gbq, create_doc_final, open_init_data,
    execute_with_context,
    init_group_id, 
    deal_merged_places, 
    deal_unmerged_places, 
    choose_prefered_document, 
    create_agrupamento_inspecao, 
    grouped_subs_asterisk, 
    final_ajustes)

In [64]:
def execute(func, df, display_flag=True, verbose=True, LEVEL_GROUP=None, use_level=False, **kwargs):
    """
    Executes a given function on the DataFrame, optionally displays the result, and shows execution time.

    Parameters:
    - func: The processing function to execute. It should accept a DataFrame and return a DataFrame.
    - df: The DataFrame to process.
    - display_flag: Boolean flag indicating whether to display the DataFrame after processing.
    - verbose: Boolean flag indicating whether to print execution details.
    - LEVEL_GROUP: Optional parameter to pass a level group to the function.
    - use_level: Boolean flag indicating whether to use the LEVEL_GROUP parameter.
    - **kwargs: Additional keyword arguments to pass to the function.

    Returns:
    - The processed DataFrame.
    """

    start_time = time.time()  # Start timing before function execution
  

    if verbose:
        print(f'_______________\nExecuting function {func.__name__}...')

    if use_level:
        df = func(df, LEVEL_GROUP=LEVEL_GROUP, **kwargs)
    else:
        df = func(df, **kwargs)

    end_time = time.time()  # End timing after function execution
    execution_time = end_time - start_time  # Calculate execution time

    if verbose:
        print(f'Function {func.__name__} executed successfully in {execution_time:.2f} seconds.')
    
    
      
    if 'group_id' in df.columns:
        s = df['group_id'].str.len()
        print(f'Result Max group id size: {s.max()}')
        
        display(df.groupby('group_id').agg({
            #'group_id': 'nunique',
            'merchant_market_hierarchy_id': 'nunique'
        }).max())
        
        
    if 'group_id_index' in df.columns:
        sizes = df.groupby('group_id_index').ngroups
        print(f'qtd names group id: {sizes}')
        
    
    if 'group_id' in df.columns:
        #sizes = df['group_id_index'].value_counts().values[0]
        #print(f'qtd names group id: {sizes}')
        print('Group ID in cols')
        big_group = df['group_id'].value_counts().index[0]
        display(df[df['group_id']==big_group])
    
    #if 'group_id' in df.columns: print(f'Number of groups: {df["group_id"].nunique()}')
    
    if display_flag:
        display(df)
    return df


def execute_with_context(LEVEL_GROUP, display_flag=False, verbose=False):
    import inspect
    def execute_context(func, df, **kwargs):
        # Check if 'LEVEL_GROUP' is in the function's parameters
        params = inspect.signature(func).parameters
        if 'LEVEL_GROUP' in params:
            # If 'LEVEL_GROUP' is a parameter, pass it along with 'df'
            result = execute(func, df, display_flag=display_flag, verbose=verbose, LEVEL_GROUP=LEVEL_GROUP,
                             use_level=True, **kwargs)
        else:
            # If 'LEVEL_GROUP' is not a parameter, call the function without it
            result = execute(func, df, display_flag=display_flag, verbose=verbose, use_level=False, **kwargs)
        return result
    return execute_context

In [71]:
def main_data_treat_muni(df, file_name):

    """
    Main function to process data with options to display intermediate DataFrames and print verbose messages.

    Parameters:
    - read_data_func: Function to read initial data.
    - display_flag: Boolean flag indicating whether to display the DataFrame after each processing step.
    - verbose: Boolean flag indicating whether to print verbose messages during processing.
    """
    display_flag=False
    
    len_df = len(df)
    
    #if len_df> 100000:
    #    verbose = True
    #else:
    #    verbose = False
    verbose = True
    LEVEL_GROUP = ['cod_muni']
    
    
    print(f'Starting main data treat for file {file_name}, with size {len(df)}')

    start = time.time()
    execute_ctx = execute_with_context(LEVEL_GROUP, display_flag=display_flag, verbose=verbose)

    df = execute_ctx(init_group_id, df)
    #df = execute_ctx(deal_unmerged_places, df)
    #df = execute_ctx(deal_merged_places, df)
    
    #df = execute_ctx(choose_prefered_document, df)
    #df = execute_ctx(create_agrupamento_inspecao, df)
    #df = execute_ctx(grouped_subs_asterisk, df)
    #df = execute_ctx(final_ajustes, df)
    #df = execute_ctx(create_doc_final, df)
    
    print(f'Main data treat Execution time file {file_name}: {time.time() - start:.2f} seconds')
    return df

In [72]:
df3 = main_data_treat_muni(df, 'teste SP')

Starting main data treat for file teste SP, with size 1334905
_______________
Executing function init_group_id...
Function init_group_id executed successfully in 15.50 seconds.
Result Max group id size: 115


merchant_market_hierarchy_id    472
dtype: int64

Group ID in cols


,subs_asterisk,nome_master,nome_muni,uf,cpf,cpf_brasil,qtd_cpfs,qtd_cnpjs,cnpj,numero_inicio,...,merchant_tax_id,mmhid_merge,merchant_market_hierarchy_id,id_ton,nome_ton,inicio,grouped_names,agrupamento_nome_1,_merge,group_id
139937,Outros,MARIAAPARE,São Paulo,SP,27304660880,None,14270,2771,30284929000150,50564034,...,50564034000187,867636597,867636597,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,MARIAAPARECIDA|São Paulo SP
139938,Outros,MARIAAPARE,São Paulo,SP,27304660880,None,14270,2771,30284929000150,52451250,...,52451250000141,835644909,835644909,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,MARIAAPARECIDA|São Paulo SP
139939,Outros,MARIAAPARE,São Paulo,SP,27304660880,None,14270,2771,30284929000150,None,...,None,<NA>,283762615,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,MARIAAPARECIDA|São Paulo SP
139940,Outros,MARIAAPARE,São Paulo,SP,27304660880,None,14270,2771,30284929000150,54311508,...,54311508000130,860844383,860844383,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,MARIAAPARECIDA|São Paulo SP
139941,Outros,MARIAAPARE,São Paulo,SP,27304660880,None,14270,2771,30284929000150,53767821,...,53767821000114,855447427,855447427,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,MARIAAPARECIDA|São Paulo SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205356,Outros_SumUp,MARIAAPARECIDA,São Paulo,SP,32646176848,None,14261,2771,22971232000120,None,...,06167186000154,316535235,316535235,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,MARIAAPARECIDA|São Paulo SP
715544,MercadoPago,MARIAAPARECID,São Paulo,SP,16067848899,None,14262,2771,42511745000165,None,...,None,<NA>,<NA>,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,MARIAAPARECIDA|São Paulo SP
715545,Outros_Pags,MARIAAPARECID,São Paulo,SP,16067848899,None,14262,2771,42511745000165,None,...,,817833689,817833689,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,MARIAAPARECIDA|São Paulo SP
715546,Outros_Pags,MARIAAPARECID,São Paulo,SP,16067848899,None,14262,2771,42511745000165,None,...,13879618000115,526482023,526482023,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,MARIAAPARECIDA|São Paulo SP


Main data treat Execution time file teste SP: 22.00 seconds


In [88]:


def deal_unmerged_places(df, LEVEL_GROUP):
    df = df.copy()

    df['mmhid_places'] = df['mmhid_merge'].apply(lambda x: str(x) if not pd.isna(x) else np.nan) #deal with nan in Int
    df['new_id'] = np.arange(df.shape[0])
    df.loc[df['subs_asterisk']=='Ton', 'mmhid_places'] = 'Ton' + df['id_ton'].astype(str)
    df.loc[df['mmhid_places'].isnull(), 'mmhid_places'] = 'created' + df['new_id'].astype(str)

    #df['muni'] =  df[LEVEL_GROUP].astype(str).apply(lambda x: ' '.join(x))

    if len(LEVEL_GROUP)>1:
        df['mmhid_places'] = df['mmhid_places'] + ' - ' + df[LEVEL_GROUP].astype(str).apply(lambda x: '|'.join(x), axis=1)#df['muni']
    elif len(LEVEL_GROUP)==1:
        df['mmhid_places'] = df['mmhid_places'] + ' - ' + df[LEVEL_GROUP[0]].astype(str)
        
    df = assign_group_ids(df, ['group_id'], final_col='group_id_idx1')
        
    df['tam_group1'] = df.groupby('group_id_idx1')['nome_master'].transform('count')


    df = assign_group_ids(df, ['group_id', 'mmhid_places'], final_col='group_id_idx2')
    
    df['tam_group2'] = df.groupby('group_id_idx2')['nome_master'].transform('count')
    
    #df_grouped = df.copy()
    # SPLIT group_id by | . Get first element
    #df_grouped['group_id'] = df_grouped['group_id'].str.split('|').apply(lambda x: x[0])

    # concat group_id with group_id_index
    #df_grouped = df_grouped.groupby(['group_id_index'], as_index=False)['group_id'].apply(lambda x: list(x.dropna().unique()))
    #df_grouped = df_grouped.groupby(['group_id_index'], as_index=False)['group_id'].apply(lambda x: list(x.dropna().unique()))
    #df_grouped = df_grouped[['group_id_index', 'group_id']].drop_duplicates().groupby('group_id_index', as_index=False).head(10)
    
    #df_grouped = df_grouped.groupby(['group_id_index'], as_index=False)['group_id'].apply(lambda x: ', '.join(x))

    #df = df.drop(columns=['group_id', 'mmhid_places', 'new_id', 'id_ton'])
    #df = df.merge(df_grouped, on='group_id_index', how='left')

    #df = df.drop(columns=['group_id_index'])
    return df

df4 = deal_unmerged_places(df3, LEVEL_GROUP=[])
#df4['len_resultado'] = df4.groupby('group_id_index')['nome_master'].transform('count')
#df4[df4['len_resultado']>1000]['agrupamento_nome_1'].unique()

In [89]:
def deal_merged_places(df, LEVEL_GROUP):
    df = df.copy()

    
    df['mmhid_places'] = df.loc[df['subs_asterisk'].isin(OUTROS), 'merchant_market_hierarchy_id']

    df_grouped = df.copy()

    level_group2 = ['group_id_idx2']
    
    df_grouped = df_grouped[df_grouped['mmhid_places'].notnull()]
    df_grouped = df_grouped[LEVEL_GROUP + level_group2 +  ['mmhid_places']].drop_duplicates()
    # Group by the specified columns and aggregate
    df_grouped = df_grouped.groupby(LEVEL_GROUP + level_group2).agg(
        unique_mmhid_places=('mmhid_places', 'first'),
        nunique_mmhid_places=('mmhid_places', 'nunique')
    ).reset_index()

    df = df.merge(df_grouped, on=LEVEL_GROUP + level_group2, how='left')
    
    df['group_id_idx3'] = df['group_id_idx2']
    

    df.loc[(df['nunique_mmhid_places']>1)
           & (df['subs_asterisk'].isin(OUTROS))
           , 'group_id'] = df['group_id_idx3'].astype(str) + ' - ' + df['mmhid_places'].astype(str)
    
    df.loc[(df['nunique_mmhid_places']>1)
           & (~df['subs_asterisk'].isin(OUTROS))
           , 'group_id'] = df['group_id_idx3'].astype(str) + ' - ' + df['unique_mmhid_places'].astype(str)

    
    df = assign_group_ids(df, ['group_id_idx3'], final_col='group_id_idx3')
    
    df = df.drop(columns=['mmhid_places', 'unique_mmhid_places', 'nunique_mmhid_places'])
    df['tam_group3'] = df.groupby('group_id_idx3')['nome_master'].transform('count')
    
    return df


In [92]:
OUTROS = ['Outros', 'Outros_Pags', 'Outros_SumUp']
df4 = deal_merged_places(df4, LEVEL_GROUP=['cod_muni'])

In [77]:
df4[df4['len_resultado']>1000]

,subs_asterisk,nome_master,nome_muni,uf,cpf,cpf_brasil,qtd_cpfs,qtd_cnpjs,cnpj,numero_inicio,...,nome_ton,inicio,grouped_names,agrupamento_nome_1,_merge,group_id,mmhid_places,new_id,group_id_index,len_resultado
1855,Ifood,RESTAURANTEYAMASU,São Paulo,SP,None,None,<NA>,1,29790794000116,None,...,None,RESTAU,"[RESTAURANTEYAMASU, RESTAURANTEYAMASUSHI]","RESTAURANTEYAMASU, RESTAURANTEYAMASUSHI3550308",both,RESTAURANTEYAMASUSHI|São Paulo SP,created1855,1855,864830,1720
2026,Ifood,CARLOS,São Paulo,SP,51223249808,None,49359,11723,53140292000124,52561332,...,None,CARLOS,[CARLOS],"CARLOS23136047800, CARLOS3550308",both,CARLOS|São Paulo SP,created2026,2026,864830,1720
2027,Outros,CARLOS,São Paulo,SP,51223249808,None,49359,11723,53140292000124,None,...,None,CARLOS,[CARLOS],"CARLOS23136047800, CARLOS3550308",both,CARLOS|São Paulo SP,447788095,2027,864830,1720
2028,Outros,CARLOS,São Paulo,SP,51223249808,None,49359,11723,53140292000124,None,...,None,CARLOS,[CARLOS],"CARLOS23136047800, CARLOS3550308",both,CARLOS|São Paulo SP,833569798,2028,864830,1720
2029,Outros,CARLOS,São Paulo,SP,51223249808,None,49359,11723,53140292000124,None,...,None,CARLOS,[CARLOS],"CARLOS23136047800, CARLOS3550308",both,CARLOS|São Paulo SP,583736683,2029,864830,1720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305786,Ton,LANCHONETERESTAURANTEMAEEFILHO,São Paulo,SP,11127752855,None,<NA>,<NA>,None,None,...,LANCHONETE RESTAURANTE MAE E FILHO,LANCHO,"[LANCHONETERESTAURANTEMAEEFILHO, LANCHONETERES...","LANCHONETERESTAURANTEMAEEFILHO, LANCHONETEREST...",both,LANCHONETERESTAURANTEMAEEFILHO|São Paulo SP,Ton1629278.0,1305786,864830,1720
1306322,Ton,EMPORIOCOLINAS,São Paulo,SP,08896698804,None,<NA>,<NA>,None,None,...,EMPORIO COLINAS,EMPORI,"[EMPORIOCOLINAS, EMPORIOCOLINA]","EMPORIOCOLINAS, EMPORIOCOLINA3550308",both,EMPORIOCOLINAS|São Paulo SP,Ton1632976.0,1306322,864830,1720
1319152,Ton,HAYAIRESTAURANTE,São Paulo,SP,None,None,<NA>,<NA>,10991077000124,None,...,HAYAI RESTAURANTE,HAYAIR,[HAYAIRESTAURANTE],HAYAIRESTAURANTE3550308,both,HAYAIRESTAURANTE|São Paulo SP,Ton1716459.0,1319152,864830,1720
1321180,Ton,SBTECHINFORMATICA,São Paulo,SP,33149421845,None,<NA>,<NA>,None,None,...,SBTECH INFORMATICA,SBTECH,[SBTECHINFORMATICA],SBTECHINFORMATICA3550308,both,SBTECHINFORMATICA|São Paulo SP,Ton1729500.0,1321180,864830,1720


In [83]:
df4[df4['group_id']=='RESTAURANTEYAMASUSHI|São Paulo SP'][['nome_master', 'group_id', 'agrupamento_nome_1', 'mmhid_places', 'group_id_index']]

,nome_master,group_id,agrupamento_nome_1,mmhid_places,group_id_index
1855,RESTAURANTEYAMASU,RESTAURANTEYAMASUSHI|São Paulo SP,"RESTAURANTEYAMASU, RESTAURANTEYAMASUSHI3550308",created1855,864830
257737,RESTAURANTEYAMASUSHI,RESTAURANTEYAMASUSHI|São Paulo SP,"RESTAURANTEYAMASU, RESTAURANTEYAMASUSHI3550308",505114550,864830


In [85]:
df4[df4['mmhid_places']=='505114550'][['nome_master', 'group_id', 'agrupamento_nome_1', 'mmhid_places', 'group_id_index']]

,nome_master,group_id,agrupamento_nome_1,mmhid_places,group_id_index
8364,RESTAURANTE,RESTAURANTE|São Paulo SP,"RESTAURANTE, RESTAURANTE041946898603550308",505114550,864830
257737,RESTAURANTEYAMASUSHI,RESTAURANTEYAMASUSHI|São Paulo SP,"RESTAURANTEYAMASU, RESTAURANTEYAMASUSHI3550308",505114550,864830


In [86]:
df4[df4['group_id']=='RESTAURANTE|São Paulo SP'][['nome_master', 'group_id', 'agrupamento_nome_1', 'mmhid_places', 'group_id_index']]

,nome_master,group_id,agrupamento_nome_1,mmhid_places,group_id_index
8339,RESTAURANTE,RESTAURANTE|São Paulo SP,"RESTAURANTE, RESTAURANTE041946898603550308",647431069,864830
8340,RESTAURANTE,RESTAURANTE|São Paulo SP,"RESTAURANTE, RESTAURANTE041946898603550308",815781288,864830
8341,RESTAURANTE,RESTAURANTE|São Paulo SP,"RESTAURANTE, RESTAURANTE041946898603550308",866201937,864830
8342,RESTAURANTE,RESTAURANTE|São Paulo SP,"RESTAURANTE, RESTAURANTE041946898603550308",799200892,864830
8343,RESTAURANTE,RESTAURANTE|São Paulo SP,"RESTAURANTE, RESTAURANTE041946898603550308",763948377,864830
...,...,...,...,...,...
8518,RESTAURANTE,RESTAURANTE|São Paulo SP,"RESTAURANTE, RESTAURANTE041946898603550308",715575740,864830
8519,RESTAURANTE,RESTAURANTE|São Paulo SP,"RESTAURANTE, RESTAURANTE041946898603550308",565315603,864830
8520,RESTAURANTE,RESTAURANTE|São Paulo SP,"RESTAURANTE, RESTAURANTE041946898603550308",847270011,864830
8521,RESTAURANTE,RESTAURANTE|São Paulo SP,"RESTAURANTE, RESTAURANTE041946898603550308",539157007,864830


In [68]:
df3['len_resultado'] = df3.groupby('group_id')['nome_master'].transform('count')

In [69]:
df3

,subs_asterisk,nome_master,nome_muni,uf,cpf,cpf_brasil,qtd_cpfs,qtd_cnpjs,cnpj,numero_inicio,...,mmhid_merge,merchant_market_hierarchy_id,nome_ton,inicio,grouped_names,agrupamento_nome_1,_merge,group_id_index,group_id,len_resultado
0,SumUp,REYTECH,São Paulo,SP,None,None,<NA>,1,55288394000117,None,...,<NA>,<NA>,None,REYTEC,[REYTECH],REYTECH3550308,both,1,REYTECH|São Paulo SP,1
1,SumUp,APPSERVIDORE,São Paulo,SP,None,None,<NA>,<NA>,None,None,...,<NA>,<NA>,None,APPSER,[APPSERVIDORE],APPSERVIDORE3550308,both,2,APPSERVIDORE|São Paulo SP,1
2,SumUp,JOSEWILSONP,São Paulo,SP,17782026830,None,26,10,20701303000149,None,...,<NA>,<NA>,None,JOSEWI,[JOSEWILSONP],"JOSEWILSONP, JOSEWILSONPEGADO3550308",both,3,JOSEWILSONP|São Paulo SP,1
3,SumUp,MARIAARTS,São Paulo,SP,None,None,<NA>,<NA>,None,None,...,<NA>,<NA>,None,MARIAA,[MARIAARTS],MARIAARTS3550308,both,4,MARIAARTS|São Paulo SP,1
4,SumUp,CANILBUTDOGG,São Paulo,SP,None,None,<NA>,<NA>,None,None,...,<NA>,<NA>,None,CANILB,[CANILBUTDOGG],CANILBUTDOGG3550308,both,5,CANILBUTDOGG|São Paulo SP,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334900,Ton,CLINICAODONTOLOGIADENISEFERES,São Paulo,SP,None,None,<NA>,<NA>,03670613000124,None,...,<NA>,<NA>,CLINICA ODONTOLOGIA DENISE FERES,CLINIC,[CLINICAODONTOLOGIADENISEFERES],"CLINICAODONTOLOGIA, CLINICAODONTOLOGIADENISEFE...",both,878144,CLINICAODONTOLOGICADENISEFERESLTDA|São Paulo S...,2
1334901,Ton,POINTDOCOYOTE,São Paulo,SP,16154481870,None,<NA>,<NA>,None,None,...,<NA>,<NA>,POINT DO COYOTE,POINTD,[POINTDOCOYOTE],POINTDOCOYOTE3550308,both,878145,"VANDERLEIBATISTABARROS|São Paulo SP, POINTDOCO...",2
1334902,Ton,ANGELICADAMOTALIMA,São Paulo,SP,44972567821,None,<NA>,<NA>,None,None,...,<NA>,<NA>,ANGELICA DA MOTA LIMA,ANGELI,[ANGELICADAMOTALIMA],ANGELICADAMOTALIMA3550308,both,878146,"ANGLICADAMOTALIMA|São Paulo SP, ANGELICADAMOTA...",2
1334903,Ton,BRUNACECCHETTIESTETICA,São Paulo,SP,39824084827,None,<NA>,<NA>,None,None,...,<NA>,<NA>,BRUNA CECCHETTI ESTETICA,BRUNAC,[BRUNACECCHETTIESTETICA],BRUNACECCHETTIESTETICA3550308,both,594277,"BRUNACECCHETTIMENDESDASILVA|São Paulo SP, BRUN...",3


In [70]:
df3[df3['len_resultado']>1000]['agrupamento_nome_1'].unique()

array(['RESTAURANTEYAMASU, RESTAURANTEYAMASUSHI3550308',
       'CARLOS23136047800, CARLOS3550308',
       'RESTAURANTED, RESTAURANTEDAANA, RESTAURANTEDA, RESTAURANTEDAA3550308',
       'RESTAURANTEFRIGID, RESTAURANTEFRIGIDEIRA, RESTAURANTEFRIGIDEIR3550308',
       'SABORNORDESTINO, SABORNORDESTI, SABORNORDESTINODALU, SABORNORDESTIN3550308',
       'RCDESOUSA, RCDESOUSAALIMENTO, RCDESOUSAALIME3550308',
       'RJGOMESLANCHON, RJGOMESLANCHONETEE3550308',
       'RESTAURANTE, RESTAURANTE041946898603550308', 'PERAEACAI3550308',
       'BARDALOIRA10152151818, BARDALOIRA3550308',
       'MARIORESTAURANTE, MARIORESTAUR3550308',
       'RESTAURANTEEPIZZAR, RESTAURANTEEPIZZARIAKINJOSLTDA, RESTAURANTEEPIZZARI, RESTAURANTEEPIZZ, RESTAURANTEEPIZZA, RESTAURANTEEPIZZARIA3550308',
       'LANCHSANTAVITORIA3550308',
       'LANCHONETEEACAIDOR, LANCHONETEEACA, LANCHONETEEACAI, LANCHONETEE, LANCHONETEEA3550308',
       'RESTAURANTEGAL, RESTAURANTEGA, RESTAURANTEGALERI, RESTAURANTEGALERIA3550308',
     

In [56]:
df3


,subs_asterisk,nome_master,nome_muni,uf,cpf,cpf_brasil,qtd_cpfs,qtd_cnpjs,cnpj,numero_inicio,...,merchant_tax_id,mmhid_merge,merchant_market_hierarchy_id,nome_ton,inicio,grouped_names,agrupamento_nome_1,_merge,group_id_index,group_id
0,SumUp,REYTECH,São Paulo,SP,None,None,<NA>,1,55288394000117,None,...,None,<NA>,<NA>,None,REYTEC,[REYTECH],REYTECH3550308,both,1,REYTECH|São Paulo SP
1,SumUp,APPSERVIDORE,São Paulo,SP,None,None,<NA>,<NA>,None,None,...,None,<NA>,<NA>,None,APPSER,[APPSERVIDORE],APPSERVIDORE3550308,both,2,APPSERVIDORE|São Paulo SP
2,SumUp,JOSEWILSONP,São Paulo,SP,17782026830,None,26,10,20701303000149,None,...,None,<NA>,<NA>,None,JOSEWI,[JOSEWILSONP],"JOSEWILSONP, JOSEWILSONPEGADO3550308",both,3,JOSEWILSONP|São Paulo SP
3,SumUp,MARIAARTS,São Paulo,SP,None,None,<NA>,<NA>,None,None,...,None,<NA>,<NA>,None,MARIAA,[MARIAARTS],MARIAARTS3550308,both,4,MARIAARTS|São Paulo SP
4,SumUp,CANILBUTDOGG,São Paulo,SP,None,None,<NA>,<NA>,None,None,...,None,<NA>,<NA>,None,CANILB,[CANILBUTDOGG],CANILBUTDOGG3550308,both,5,CANILBUTDOGG|São Paulo SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334900,Ton,CLINICAODONTOLOGIADENISEFERES,São Paulo,SP,None,None,<NA>,<NA>,03670613000124,None,...,None,<NA>,<NA>,CLINICA ODONTOLOGIA DENISE FERES,CLINIC,[CLINICAODONTOLOGIADENISEFERES],"CLINICAODONTOLOGIA, CLINICAODONTOLOGIADENISEFE...",both,878144,CLINICAODONTOLOGICADENISEFERESLTDA|São Paulo S...
1334901,Ton,POINTDOCOYOTE,São Paulo,SP,16154481870,None,<NA>,<NA>,None,None,...,None,<NA>,<NA>,POINT DO COYOTE,POINTD,[POINTDOCOYOTE],POINTDOCOYOTE3550308,both,878145,"VANDERLEIBATISTABARROS|São Paulo SP, POINTDOCO..."
1334902,Ton,ANGELICADAMOTALIMA,São Paulo,SP,44972567821,None,<NA>,<NA>,None,None,...,None,<NA>,<NA>,ANGELICA DA MOTA LIMA,ANGELI,[ANGELICADAMOTALIMA],ANGELICADAMOTALIMA3550308,both,878146,"ANGLICADAMOTALIMA|São Paulo SP, ANGELICADAMOTA..."
1334903,Ton,BRUNACECCHETTIESTETICA,São Paulo,SP,39824084827,None,<NA>,<NA>,None,None,...,None,<NA>,<NA>,BRUNA CECCHETTI ESTETICA,BRUNAC,[BRUNACECCHETTIESTETICA],BRUNACECCHETTIESTETICA3550308,both,594277,"BRUNACECCHETTIMENDESDASILVA|São Paulo SP, BRUN..."


In [35]:
df2 = pd.read_parquet('data/nomes_agrupados/part_3550308.parquet')
df2['len_resultado'] = df2.groupby(['agrupamento_nome_1'])['nome_master'].transform('count')
df2

,nome_master,inicio,cod_muni,resultado_names,agrupamento_nome_1,len_resultado
0,,,3550308,[],,1
1,AAAA,AAAA,3550308,[AAAA],AAAA,1
2,AAAAAAAAAAAA,AAAAAA,3550308,[AAAAAAAAAAAA],AAAAAAAAAAAA,1
3,AAAAAAAAAAOKINAWA,AAAAAA,3550308,[AAAAAAAAAAOKINAWA],AAAAAAAAAAOKINAWA,1
4,AAAAAADOMONGE,AAAAAA,3550308,[AAAAAADOMONGE],AAAAAADOMONGE,1
...,...,...,...,...,...,...
1095648,ZZDISTRIBUIDORADEGLP,ZZDIST,3550308,[ZZDISTRIBUIDORADEGLP],ZZDISTRIBUIDORADEGLP,1
1095649,ZZJOIAS,ZZJOIA,3550308,[ZZJOIAS],ZZJOIAS,1
1095650,ZZMATPARA,ZZMATP,3550308,[ZZMATPARA],ZZMATPARA,1
1095651,ZZNSHOESCALCADOSE,ZZNSHO,3550308,[ZZNSHOESCALCADOSE],ZZNSHOESCALCADOSE,1


In [37]:
df2.sort_values('len_resultado')

,nome_master,inicio,cod_muni,resultado_names,agrupamento_nome_1,len_resultado
0,,,3550308,[],,1
663321,MARCELORAYMUNDODEOL,MARCEL,3550308,[MARCELORAYMUNDODEOL],MARCELORAYMUNDODEOL,1
663322,MARCELOREBEQUI,MARCEL,3550308,[MARCELOREBEQUI],MARCELOREBEQUI,1
663323,MARCELOREGIANI,MARCEL,3550308,[MARCELOREGIANI],MARCELOREGIANI,1
663326,MARCELOREGISFERREIRA,MARCEL,3550308,[MARCELOREGISFERREIRA],MARCELOREGISFERREIRA,1
...,...,...,...,...,...,...
700533,MATERIALDE,MATERI,3550308,"[MATERIALDEC, MATERIALDECONSTRUCA, MATERIALDE,...","MATERIALDEC, MATERIALDECONSTRUCAO01488093598, ...",10
700534,MATERIALDEC,MATERI,3550308,"[MATERIALDEC, MATERIALDECONSTRUCA, MATERIALDE,...","MATERIALDEC, MATERIALDECONSTRUCAO01488093598, ...",10
700535,MATERIALDECO,MATERI,3550308,"[MATERIALDEC, MATERIALDECONSTRUCA, MATERIALDE,...","MATERIALDEC, MATERIALDECONSTRUCAO01488093598, ...",10
700537,MATERIALDECONS,MATERI,3550308,"[MATERIALDEC, MATERIALDECONSTRUCA, MATERIALDE,...","MATERIALDEC, MATERIALDECONSTRUCAO01488093598, ...",10


In [34]:
df2[df2['agrupamento_nome_1']=='CLAUDIAAPARECIDATANI']

,nome_master,inicio,cod_muni,resultado_names,agrupamento_nome_1,len_resultado
204942,CLAUDIAAPARECIDATANI,CLAUDI,3550308,[CLAUDIAAPARECIDATANI],CLAUDIAAPARECIDATANI,486


In [38]:
df

,subs_asterisk,nome_master,nome_muni,uf,cpf,cpf_brasil,qtd_cpfs,qtd_cnpjs,cnpj,numero_inicio,cod_muni,merchant_tax_id,mmhid_merge,merchant_market_hierarchy_id,id_ton,nome_ton,inicio,resultado_names,agrupamento_nome_1,_merge
0,SumUp,REYTECH,São Paulo,SP,None,None,<NA>,1,55288394000117,None,3550308,None,<NA>,<NA>,NaN,None,REYTEC,[REYTECH],REYTECH3550308,both
1,SumUp,APPSERVIDORE,São Paulo,SP,None,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,NaN,None,APPSER,[APPSERVIDORE],APPSERVIDORE3550308,both
2,SumUp,JOSEWILSONP,São Paulo,SP,17782026830,None,26,10,20701303000149,None,3550308,None,<NA>,<NA>,NaN,None,JOSEWI,[JOSEWILSONP],"JOSEWILSONP, JOSEWILSONPEGADO3550308",both
3,SumUp,MARIAARTS,São Paulo,SP,None,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,NaN,None,MARIAA,[MARIAARTS],MARIAARTS3550308,both
4,SumUp,CANILBUTDOGG,São Paulo,SP,None,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,NaN,None,CANILB,[CANILBUTDOGG],CANILBUTDOGG3550308,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334900,Ton,CLINICAODONTOLOGIADENISEFERES,São Paulo,SP,None,None,<NA>,<NA>,03670613000124,None,3550308,None,<NA>,<NA>,1818529.0,CLINICA ODONTOLOGIA DENISE FERES,CLINIC,[CLINICAODONTOLOGIADENISEFERES],"CLINICAODONTOLOGIA, CLINICAODONTOLOGIADENISEFE...",both
1334901,Ton,POINTDOCOYOTE,São Paulo,SP,16154481870,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,1818530.0,POINT DO COYOTE,POINTD,[POINTDOCOYOTE],POINTDOCOYOTE3550308,both
1334902,Ton,ANGELICADAMOTALIMA,São Paulo,SP,44972567821,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,1818554.0,ANGELICA DA MOTA LIMA,ANGELI,[ANGELICADAMOTALIMA],ANGELICADAMOTALIMA3550308,both
1334903,Ton,BRUNACECCHETTIESTETICA,São Paulo,SP,39824084827,None,<NA>,<NA>,None,None,3550308,None,<NA>,<NA>,1818555.0,BRUNA CECCHETTI ESTETICA,BRUNAC,[BRUNACECCHETTIESTETICA],BRUNACECCHETTIESTETICA3550308,both


In [39]:
df['len_resultado'] = df.groupby(['agrupamento_nome_1'])['nome_master'].transform('count')

In [40]:
df.sort_values('len_resultado')

,subs_asterisk,nome_master,nome_muni,uf,cpf,cpf_brasil,qtd_cpfs,qtd_cnpjs,cnpj,numero_inicio,...,merchant_tax_id,mmhid_merge,merchant_market_hierarchy_id,id_ton,nome_ton,inicio,resultado_names,agrupamento_nome_1,_merge,len_resultado
0,SumUp,REYTECH,São Paulo,SP,None,None,<NA>,1,55288394000117,None,...,None,<NA>,<NA>,NaN,None,REYTEC,[REYTECH],REYTECH3550308,both,1
831357,Outros_Pags,ABXGKS,São Paulo,SP,None,None,<NA>,<NA>,None,None,...,None,<NA>,811272556,NaN,None,ABXGKS,[ABXGKS],ABXGKS3550308,both,1
831358,Outros_Pags,CRISTIANBARBOSADE,São Paulo,SP,None,None,<NA>,<NA>,None,None,...,39451476000184,706622354,706622354,NaN,None,CRISTI,[CRISTIANBARBOSADE],CRISTIANBARBOSADE3550308,both,1
831359,Outros_Pags,JACBOUTIQUE,São Paulo,SP,None,None,<NA>,<NA>,None,None,...,49990855000104,796744485,796744485,NaN,None,JACBOU,[JACBOUTIQUE],JACBOUTIQUE3550308,both,1
831360,Outros_Pags,ROSIMARDEJESUSDOS,São Paulo,SP,32804434826,None,1,<NA>,None,None,...,,774904701,774904701,NaN,None,ROSIMA,[ROSIMARDEJESUSDOS],ROSIMARDEJESUSDOS3550308,both,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204950,Outros_Pags,MARIAAPARECIDA,São Paulo,SP,32646176848,None,14261,2771,22971232000120,None,...,,856914648,856914648,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,486
205171,Outros_Pags,MARIAAPARECIDA,São Paulo,SP,32646176848,None,14261,2771,22971232000120,None,...,,820486249,820486249,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,486
205170,Outros_Pags,MARIAAPARECIDA,São Paulo,SP,32646176848,None,14261,2771,22971232000120,None,...,08561701000101,606337622,606337622,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,486
205175,Outros_Pags,MARIAAPARECIDA,São Paulo,SP,32646176848,None,14261,2771,22971232000120,None,...,,644081404,644081404,NaN,None,MARIAA,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",both,486


In [41]:
df2[df2['nome_master']=='MARIAAPARECIDA']

,nome_master,inicio,cod_muni,resultado_names,agrupamento_nome_1,len_resultado
674829,MARIAAPARECIDA,MARIAA,3550308,"[MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MA...","MARIAAPARECID, MARIAAPARECIDA, MARIAAPARE, MAR...",5
